In [1]:
import argparse
import json
from pathlib import Path
import statistics

from EvalWhisper import EvalWhisper



/home/dummy/Desktop/asr_project/salASR/salasr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from tqdm import tqdm



In [21]:
def calculate_stats(scores_list):
    return statistics.mean(scores_list), statistics.stdev(scores_list)

def dump_output(scores_list, mean, standard_deviation, output_path):
    output_dict = {}
    output_dict["mean"] = mean
    output_dict["standard_deviation"] = standard_deviation
    output_dict["scores"] = scores_list
    with open(output_path, "w") as output_file:
        json.dump(output_dict, output_file)

def main(args):
    # parse args
    # args = parser.parse_args()
    num_samples = 20
    model_size = "tiny"
    output_dir = Path("./")
    output_file = "output_retain_top_0.8.txt"
    r_value = 0.8
    mode_value = "retain"
    what_to_mask = "top"
    #load processor and model
    print(f"Loading model . . . ")
    whisper_evaluator = EvalWhisper(f"openai/whisper-{model_size}")
    print(f"Loaded model")
    ds = load_dataset("librispeech_asr", split="validation.clean", streaming=True)
    print(f"Loaded data")
    scores_list = []
    with open(output_dir / output_file, "a") as score_writer:
        for sample in tqdm(ds.take(num_samples)):
            score = whisper_evaluator.evaluate(sample, whisper_evaluator.top_r_features(sample, r=r_value, mode=mode_value, where=what_to_mask))
            scores_list.append(score)
            score_writer.write(str(score) + "\n")
    print(f"scores_list:{scores_list}")
    output_path = output_dir / "outputretain_top_0.8.json"
    mean, standard_deviation = calculate_stats(scores_list)
    dump_output(scores_list, mean, standard_deviation, output_path)
    




In [22]:
main({})

Loading model . . . 
Loaded model
Loaded data


20it [01:41,  5.06s/it]

scores_list:[0.84375, 0.8235294222831726, 0.800000011920929, 0.5384615659713745, 1.0, 0.6818181872367859, 0.8095238208770752, 0.7857142686843872, 0.8181818127632141, 0.75, 0.7200000286102295, 0.8636363744735718, 0.800000011920929, 0.774193525314331, 0.7142857313156128, 0.8421052694320679, 0.7857142686843872, 0.6800000071525574, 0.8095238208770752, 0.6666666865348816]


In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-n", "--num_samples", type=int, default=10)
    parser.add_argument("-m", "--model_size", type=str, default="tiny")
    parser.add_argument("-o", "--output_dir", type=str, default="./")
    parser.add_argument("-f", "--output_file", type=str, default="output.txt")
    parser.add_argument("-r", "--r_value", type=float, default=1.0)
    parser.add_argument("--mode", type=str, default="retain")
    parser.add_argument('--mode',
                    default='retain',
                    const='retain',
                    nargs='?',
                    choices=["retain", "remove"],
                    help='arg for retaining/removing top_r feats (default: %(default)s)')
    parser.add_argument('--what',
                    default='top',
                    const='top',
                    nargs='?',
                    choices=["top", "bottom", "random"],
                    help='Specifies which part of the feature space the mask acts on to retin/remove the features (default: %(default)s)')

    args = parser.parse_args()
    main(args)